In [2]:
# imports
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


In [3]:
# load training data
df = pd.read_csv('data/training_set_VU_DM.csv')

In [ ]:
df

In [3]:
#df

pd.set_option('display.float_format', lambda x: '%.2f' % x)
df.describe().loc[['min', 'max'], :].transpose()
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
srch_id,4958347.00,166366.56,96112.23,1.00,82936.00,166507.00,249724.00,332785.00
site_id,4958347.00,9.95,7.65,1.00,5.00,5.00,14.00,34.00
visitor_location_country_id,4958347.00,175.34,65.92,1.00,100.00,219.00,219.00,231.00
visitor_hist_starrating,251866.00,3.37,0.69,1.41,2.92,3.45,3.93,5.00
visitor_hist_adr_usd,252988.00,176.02,107.25,0.00,109.81,152.24,213.49,1958.70
prop_country_id,4958347.00,173.97,68.35,1.00,100.00,219.00,219.00,230.00
prop_id,4958347.00,70079.18,40609.92,1.00,35010.00,69638.00,105168.00,140821.00
prop_starrating,4958347.00,3.18,1.05,0.00,3.00,3.00,4.00,5.00
prop_review_score,4950983.00,3.78,1.05,0.00,3.50,4.00,4.50,5.00
prop_brand_bool,4958347.00,0.63,0.48,0.00,0.00,1.00,1.00,1.00


In [5]:
df.isnull().sum().sum()
#df.columns[df.isna().any()].tolist()

119733280

In [ ]:
# number of unique values over all columns
df.nunique()

Features engineering and removal

In [12]:
# imports
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
# load training data
df = pd.read_csv('data/training_set_VU_DM.csv')

In [3]:
from functions import comp_aggregation, remove_variables, impute_linear, impute_knn

df = comp_aggregation(df)
df = remove_variables(df)
df = impute_linear(df)

In [ ]:

df.isnull().sum()

In [7]:
columns_to_check = [
    'prop_review_score',
    'prop_starrating',
    'prop_location_score1',
    'prop_location_score2',
    'prop_log_historical_price',
    'price_usd',
    'srch_length_of_stay',
    'srch_booking_window',
    'srch_adults_count',
    'srch_children_count',
    'srch_room_count',
    'orig_destination_distance',
    'comp_rate_ratio',
    'comp_inv_ratio'
]

In [ ]:
# plot boxplots
for column in columns_to_check:
    plt.figure()
    sns.boxplot(y=df[column])
    plt.title(column)
    plt.show()

In [ ]:
from scipy.stats import boxcox


for column in columns_to_check:
    # make negative values positive
    df[column] = df[column] - df[column].min() + 1
    transformed = pd.Series(boxcox(df[column])[0])
    # use iqr to remove outliers
    Q1 = transformed.quantile(0.25)
    Q3 = transformed.quantile(0.75)
    IQR = Q3 - Q1
    # use 3 for extreme outliers
    lower_bound = Q1 - 3 * IQR
    upper_bound = Q3 + 3 * IQR

    # get index of outliers in transformed
    outliers = transformed[(transformed < lower_bound) | (transformed > upper_bound)].index

    print(f'{column} has {len(outliers)} outliers')
  


In [10]:
import xgboost as xgb
from sklearn.datasets import dump_svmlight_file
from sklearn.datasets import load_svmlight_file
from sklearn.model_selection import train_test_split
from sklearn.metrics import ndcg_score

df.drop(columns="date_time", inplace=True)
df.drop(columns="gross_bookings_usd", inplace=True)

# predict position?
X = df.drop('position', axis=1)
y = df['position']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert the training data to SVMLight format
dump_svmlight_file(X_train, y_train, 'train.txt')

# Convert the testing data to SVMLight format
dump_svmlight_file(X_test, y_test, 'test.txt')

dtrain = xgb.DMatrix('train.txt')

# Set LambdaMART parameters
params = {
    'objective': 'rank:ndcg',
    'eval_metric': 'ndcg@1-5',
    'eta': 0.1,
    'max_depth': 6,
    'min_child_weight': 1,
    'gamma': 0,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'lambda': 1,
    'alpha': 0,
    'silent': 1,
    'nthread': 4
}

# Train the LambdaMART model using XGBoost
model = xgb.train(params, dtrain, num_boost_round=100)

# Load the testing data from the SVMLight file
dtest = xgb.DMatrix('test.txt')

# Make predictions using the trained LambdaMART model
y_pred = model.predict(dtest)

# Calculate NDCG score
ndcg = ndcg_score([y_test], [y_pred])

print("NDCG score:", ndcg)

# add predicted values to df
df['predicted_position'] = y_pred


Index(['srch_id', 'site_id', 'visitor_location_country_id', 'prop_country_id',
       'prop_id', 'prop_starrating', 'prop_review_score', 'prop_brand_bool',
       'prop_location_score1', 'prop_location_score2',
       'prop_log_historical_price', 'position', 'price_usd', 'promotion_flag',
       'srch_destination_id', 'srch_length_of_stay', 'srch_booking_window',
       'srch_adults_count', 'srch_children_count', 'srch_room_count',
       'srch_saturday_night_bool', 'orig_destination_distance', 'random_bool',
       'click_bool', 'gross_bookings_usd', 'booking_bool', 'comp_rate_ratio',
       'comp_inv_ratio'],
      dtype='object')
[16:31:13] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07593ffd91cd9da33-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "silent" } are not used.

NDCG score: 0.9538586752551853


In [3]:
import pandas as pd

In [4]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [5]:
our_test = pd.read_csv('data/test_set_VU_DM.csv')


In [ ]:
# common_df = pd.merge(our_test, whole, how='inner', on='date_time')

In [13]:
train[(train['srch_id'] == 3)]['prop_id']

Series([], Name: prop_id, dtype: int64)

In [11]:
our_test[our_test['srch_id'] == 3]['prop_id']

29      6533
30     11425
31     22325
32     24915
33     27753
34     32382
35     43133
36     43362
37     45550
38     46861
39     47238
40     47377
41     63039
42     64168
43     71543
44     79176
45     80734
46     90968
47     91337
48     93176
49     96614
50    119120
51    124819
52    131397
53    131940
54    131968
55    132637
Name: prop_id, dtype: int64